**텍스트 정제 => 토큰화(시퀀스 변형 x) => 불용어 제거 => 텍스트 정규화 => 표제어 추출 or 어간 추출
=> 텍스트 벡터화 => 데이터 정제**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

df = pd.read_csv('/home/kdt-admin/JY/data/drugsComTrain_raw.tsv', sep="\t", encoding='latin1')

In [ ]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,20-May-12,27.0
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,27-Apr-10,192.0
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,14-Dec-09,17.0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,03-Nov-15,10.0
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,27-Nov-16,37.0


In [ ]:
df.dropna(subset = ['condition'], inplace = True)
df.drop(columns = ['Unnamed: 0', 'date'], inplace = True)

In [ ]:
#condition컬럼에서 이상한 값 제거
df = df[~df['condition'].str.contains("users found this comment helpful.", na = False, case = False)]


**영어, 숫자만 남겨두고 제거 => 039와 관련된 단어 삭제를 위해 숫자 남김**

In [ ]:
df['review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

**소문자로 변환**

In [ ]:
df['review'] = df['review'].str.lower()
df['drugName'] = df['drugName'].str.lower()
df['condition'] = df['condition'].str.lower()

In [ ]:
df.head()

,drugName,condition,review,rating,usefulCount
0,valsartan,left ventricular dysfunction,it has no side effect i take it in combination...,9.0,27.0
1,guanfacine,adhd,my son is halfway through his fourth week of i...,8.0,192.0
2,lybrel,birth control,i used to take another oral contraceptive whic...,5.0,17.0
3,ortho evra,birth control,this is my first time using any form of birth ...,8.0,10.0
4,buprenorphine / naloxone,opiate dependence,suboxone has completely turned my life around ...,9.0,37.0


In [ ]:
df['tokenized_review'] = df['review']

# **토큰화**

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import nltk

#nltk.download('all')

In [ ]:
#nltk.download('punkt')

In [ ]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df['tokenized_review'] = df['review'].apply(tokenize_text)
df['review'] = df['review'].apply(tokenize_text)

# **불용어 제거**

**토큰화만된 리뷰와 토큰화와 불용어를 제거한 리뷰 구분**

In [ ]:
import spacy

spacy.load('en_core_web_md')
stopwords2 = spacy.lang.en.stop_words.STOP_WORDS
print(stopwords2)

{'last', '‘d', 'through', 'really', 'over', 'otherwise', 'these', 'did', 'its', 'who', 'are', 'serious', 'what', 'whereupon', 'while', 'ten', 'whereafter', 'whole', 'because', 'how', 'her', 'anyway', 'once', 'we', 'now', 'nothing', 'thence', 'fifty', 'herself', 'he', 'both', 'hundred', "'re", 'hence', 'eleven', 'hereupon', '‘s', 'around', 'itself', 'indeed', 'afterwards', 'anyhow', 'fifteen', 'doing', 'quite', 'though', 'under', 'seemed', 'throughout', 'became', 'n‘t', 'your', 'be', 'several', 'many', 'always', 'beyond', 'may', 'due', 'latter', 'per', 'our', 'my', 'becomes', 'whereas', 'others', 'yet', 'call', 'two', 'but', 'would', 'hers', 'had', 'below', 'nevertheless', 'after', 'regarding', 'out', 'thru', 'none', 'top', 'wherein', 'were', 'against', 'namely', 'where', 'further', 'still', 'everywhere', 'might', 'very', 'hereby', '‘ll', 'somewhere', 'name', 'then', 'unless', 'at', 'can', 'thus', 'upon', 'into', 'his', 'empty', 'more', 'such', 'no', 'rather', 'their', 'side', 'herein',

In [ ]:
del_list = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth',
            'week', 'weeks', 'almost', 'month', 'monthly', 'ago', 'year', 'years', 'months', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k','l', 'm', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
stopwords2 = set(stopwords2)
stop_words_list = stopwords.words('english') + del_list + list(stopwords2)
stop_words = set(stop_words_list)
updated_stopwords = stop_words

In [ ]:
def remove_stopwords(text):
    filtered_words = [word for word in text if word.lower() not in updated_stopwords]
    return filtered_words


#'가 039로 깨져있음
def remove_039(tokens):
    filtered_tokens = [token for token in tokens if not re.search(r'039', token)]
    return filtered_tokens

#영어만 남기고 삭제
english_pattern = re.compile('[a-zA-Z]+')

In [ ]:
df.head(15)

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9.0,27.0,"[it, has, no, side, effect, i, take, it, in, c..."
1,guanfacine,adhd,"[my, son, is, halfway, through, his, fourth, w...",8.0,192.0,"[my, son, is, halfway, through, his, fourth, w..."
2,lybrel,birth control,"[i, used, to, take, another, oral, contracepti...",5.0,17.0,"[i, used, to, take, another, oral, contracepti..."
3,ortho evra,birth control,"[this, is, my, first, time, using, any, form, ...",8.0,10.0,"[this, is, my, first, time, using, any, form, ..."
4,buprenorphine / naloxone,opiate dependence,"[suboxone, has, completely, turned, my, life, ...",9.0,37.0,"[suboxone, has, completely, turned, my, life, ..."
5,cialis,benign prostatic hyperplasia,"[2nd, day, on, 5mg, started, to, work, with, r...",2.0,43.0,"[2nd, day, on, 5mg, started, to, work, with, r..."
6,levonorgestrel,emergency contraception,"[he, pulled, out, but, he, cummed, a, bit, in,...",1.0,5.0,"[he, pulled, out, but, he, cummed, a, bit, in,..."
7,aripiprazole,bipolar disorde,"[abilify, changed, my, life, there, is, hope, ...",10.0,32.0,"[abilify, changed, my, life, there, is, hope, ..."
8,keppra,epilepsy,"[i, ve, had, nothing, but, problems, with, the...",1.0,11.0,"[i, ve, had, nothing, but, problems, with, the..."
9,ethinyl estradiol / levonorgestrel,birth control,"[i, had, been, on, the, pill, for, many, years...",8.0,1.0,"[i, had, been, on, the, pill, for, many, years..."


In [ ]:
df['tokenized_review'] = df['tokenized_review'].apply(remove_stopwords)
df['tokenized_review'] = df['tokenized_review'].apply(remove_039)


df['review'] = df['review'].apply(remove_039)

In [ ]:
#숫자 제거
df['review'] = df['review'].apply(lambda tokens: [token for token in tokens if english_pattern.match(token)])
df['tokenized_review'] = df['tokenized_review'].apply(lambda tokens: [token for token in tokens if english_pattern.match(token)])
#df['tokenized_review'] = df['tokenized_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

**어근 추출 : Lemmatization(표제어 추출)**

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['tokenized_review'] = df['tokenized_review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences = df['tokenized_review'], vector_size = 50, window = 5, min_count = 1, workers = 4, sg = 0)

In [ ]:
df['tokenized_review'].head()

0       [effect, combination, bystolic, mg, fish, oil]
1    [son, halfway, intuniv, concerned, began, star...
2    [oral, contraceptive, pill, cycle, happy, ligh...
3    [time, form, birth, control, glad, went, patch...
4    [suboxone, completely, turned, life, feel, hea...
Name: tokenized_review, dtype: object

In [ ]:
df.head(15)

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9.0,27.0,"[effect, combination, bystolic, mg, fish, oil]"
1,guanfacine,adhd,"[my, son, is, halfway, through, his, fourth, w...",8.0,192.0,"[son, halfway, intuniv, concerned, began, star..."
2,lybrel,birth control,"[i, used, to, take, another, oral, contracepti...",5.0,17.0,"[oral, contraceptive, pill, cycle, happy, ligh..."
3,ortho evra,birth control,"[this, is, my, first, time, using, any, form, ...",8.0,10.0,"[time, form, birth, control, glad, went, patch..."
4,buprenorphine / naloxone,opiate dependence,"[suboxone, has, completely, turned, my, life, ...",9.0,37.0,"[suboxone, completely, turned, life, feel, hea..."
5,cialis,benign prostatic hyperplasia,"[day, on, started, to, work, with, rock, hard,...",2.0,43.0,"[day, started, work, rock, hard, erection, exp..."
6,levonorgestrel,emergency contraception,"[he, pulled, out, but, he, cummed, a, bit, in,...",1.0,5.0,"[pulled, cummed, bit, took, plan, hour, later,..."
7,aripiprazole,bipolar disorde,"[abilify, changed, my, life, there, is, hope, ...",10.0,32.0,"[abilify, changed, life, hope, zoloft, clonidi..."
8,keppra,epilepsy,"[i, ve, had, nothing, but, problems, with, the...",1.0,11.0,"[problem, keppera, constant, shaking, arm, amp..."
9,ethinyl estradiol / levonorgestrel,birth control,"[i, had, been, on, the, pill, for, many, years...",8.0,1.0,"[pill, doctor, changed, rx, chateal, effective..."


# **텍스트 벡터화 or 시퀀스화 + padding**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

review1 = df['review']
review2 = df['tokenized_review']

tokenizer = Tokenizer()

tokenizer.fit_on_texts(review1)
tokenizer.fit_on_texts(review2)

review1_sequences = tokenizer.texts_to_sequences(review1)
review2_sequences = tokenizer.texts_to_sequences(review2)

max_length1 = max([len(seq) for seq in review1_sequences])
df_review1_padded = pad_sequences(review1_sequences, maxlen=max_length1, padding='post')

max_length2 = max([len(seq) for seq in review2_sequences])
df_review2_padded = pad_sequences(review2_sequences, maxlen=max_length2, padding='post')

df['review'] = df_review1_padded.tolist()

df['tokenized_review'] = df_review2_padded.tolist()

In [ ]:
df.head()

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[7, 51, 37, 47, 33, 1, 52, 7, 15, 734, 11, 266...",9.0,27.0,"[33, 734, 2667, 95, 3522, 1380, 0, 0, 0, 0, 0,..."
1,guanfacine,adhd,"[5, 568, 19, 3038, 317, 754, 2330, 103, 11, 30...",8.0,192.0,"[568, 3038, 3011, 1107, 293, 26, 22, 2113, 90,..."
2,lybrel,birth control,"[1, 203, 4, 52, 373, 1081, 1576, 117, 18, 31, ...",5.0,17.0,"[1081, 1576, 31, 452, 197, 236, 32, 1727, 12, ..."
3,ortho evra,birth control,"[14, 19, 5, 56, 24, 261, 130, 608, 11, 85, 64,...",8.0,10.0,"[24, 608, 85, 64, 522, 58, 347, 626, 730, 873,..."
4,buprenorphine / naloxone,opiate dependence,"[812, 51, 199, 700, 5, 48, 314, 1, 30, 2117, 1...",9.0,37.0,"[812, 199, 700, 48, 30, 2117, 17498, 383, 832,..."


# **drugName, condition 처리**

In [ ]:
df_1 = df.copy()

In [ ]:
#df_1[''].nunique()

In [ ]:
#값들이 밀려서 condition에 날짜값같이 이상한 값들이 있어서 삭제
#drugName에 숫자만 있는 행 제거(값이 밀림)
df_1 = df_1[~df_1['condition'].str.match('\d{2}-[a-zA-Z]{3}-\d{2}', na=False)]
df_1 = df_1[~df_1['drugName'].str.match('\d+', na=False)]

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_290073/3289786607.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_1 = df_1[~df_1['condition'].str.match('\d{2}-[a-zA-Z]{3}-\d{2}', na=False)]
/tmp/ipykernel_290073/3289786607.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_1 = df_1[~df_1['drugName'].str.match('\d+', na=False)]


In [ ]:
df_1 = df_1[df_1['usefulCount'] >= 10]

df_1 = df_1[df_1['rating'] >= 6]

In [ ]:
top_drugs = df_1['drugName'].value_counts().nlargest(20).index.tolist()
mask = df_1['drugName'].isin(top_drugs)
df_1 = df_1[mask]

In [ ]:
df_1['drug_cond'] = df_1['condition'] + '_' + df_1['drugName']

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 객체 생성 및 조건과 약물을 조합한 열에 대해 라벨 인코딩 수행
label_encoder = LabelEncoder()


# 조건에 대해 이미 생성된 라벨 인코더 객체를 사용하여 라벨 인코딩 수행
df_1['condition'] = label_encoder.fit_transform(df_1['condition'])

df_drug_encoded = pd.get_dummies(df_1['drugName'], dtype=np.int64)
df_drug_con = pd.get_dummies(df_1['drug_cond'], dtype=np.int64)

In [ ]:
df_1.head()

,drugName,condition,review,rating,usefulCount,tokenized_review,drug_cond
17,amitriptyline,28,"[29, 22, 1878, 119, 909, 35, 244, 268, 20, 665...",9.0,39.0,"[22, 1878, 909, 268, 665, 81, 527, 250, 249, 4...",ibromyalgia_amitriptyline
42,clonazepam,45,"[14, 38, 267, 5, 48, 5, 228, 512, 184, 28, 74,...",9.0,30.0,"[38, 267, 48, 228, 286, 64, 566, 167, 687, 767...",panic disorde_clonazepam
58,bupropion,66,"[178, 14, 37, 289, 2131, 76, 4549, 27, 583, 17...",10.0,15.0,"[178, 289, 430, 2628, 27, 583, 175, 199, 397, ...",smoking cessation_bupropion
79,sertraline,45,"[316, 459, 137, 17, 28, 135, 77, 1, 56, 26, 40...",10.0,28.0,"[315, 137, 26, 409, 332, 675, 124, 3029, 2833,...",panic disorde_sertraline
183,bupropion,20,"[45336, 583, 2533, 271, 6, 12, 121, 1, 30, 28,...",10.0,14.0,"[45336, 583, 2533, 271, 12, 121, 30, 59, 81, 1...",depression_bupropion


# **X, y값 분할, train, test셋 분할**

In [ ]:
#X = pd.concat([df_1[[ 'condition', 'tokenized_review']]], axis=1)
X = df_1[['condition', 'tokenized_review']]
y = df_drug_encoded

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13245 entries, 17 to 161281
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   condition         13245 non-null  int64 
 1   tokenized_review  13245 non-null  object
dtypes: int64(1), object(1)
memory usage: 310.4+ KB


In [ ]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13245 entries, 17 to 161281
Columns: 214 entries, abnormal uterine bleeding_levonorgestrel to weight loss_phentermine
dtypes: int64(214)
memory usage: 21.7 MB


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = np.array(X_train['tokenized_review'].tolist())
X_test = np.array(X_test['tokenized_review'].tolist())

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models


model = models.Sequential()
model.add(layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64))
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))  # 시퀀스 출력 반환
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(214, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
len(X_train)

10596

In [ ]:
len(y_train)

10596

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, batch_size = 128, validation_split = 0.2)

Epoch 1/50


67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.1442 - loss: 5.1855 - val_accuracy: 0.1736 - val_loss: 3.5702
Epoch 2/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2255 - loss: 3.2498 - val_accuracy: 0.3170 - val_loss: 2.4952
Epoch 3/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3565 - loss: 2.3360 - val_accuracy: 0.3722 - val_loss: 1.9760
Epoch 4/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3830 - loss: 1.9361 - val_accuracy: 0.4047 - val_loss: 1.7644
Epoch 5/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3979 - loss: 1.7521 - val_accuracy: 0.4113 - val_loss: 1.6663
Epoch 6/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4143 - loss: 1.6493 - val_accuracy: 0.4302 - val_loss: 1.6072
Epoch 7/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4247 - loss: 1.5823 - val_accuracy: 0.4292 - val_loss: 1.5718
Epoch 8/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4150 - loss: 1.5726 - val_accuracy: 0.4311 - val_loss: 1.

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

max_len = 128

input_layer = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
output = model(input_layer)
model = tf.keras.Model(inputs=input_layer, outputs=output)

# 모델 컴파일 및 훈련
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Exception encountered when calling layer 'tf_bert_for_sequence_classification_13' (type TFBertForSequenceClassification).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_for_sequence_classification_13' (type TFBertForSequenceClassification):
  • input_ids=<KerasTensor shape=(None, 128), dtype=int32, sparse=None, name=keras_tensor_252>
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • labels=None
  • training=False